In [ ]:
# mount drive first
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install timm
!pip install transformers timm ftfy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.7 MB/s eta 0:00:00


In [ ]:
def test_model(model_path, model_constructor, model_name="model"):


    model = model_constructor()
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.to(device)
    model.eval()

    preds = []
    trues = []

    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            _, predicted = torch.max(outputs, 1)

            preds.extend(predicted.cpu().numpy())
            trues.extend(labels.cpu().numpy())

    acc  = accuracy_score(trues, preds)
    prec = precision_score(trues, preds, average="weighted", zero_division=0)
    rec  = recall_score(trues, preds, average="weighted", zero_division=0)
    f1   = f1_score(trues, preds,     average="weighted", zero_division=0)

    print(f"Test Accuracy : {acc:.4f}")
    print(f"Test Precision: {prec:.4f}")
    print(f"Test Recall   : {rec:.4f}")
    print(f"Test F1 Score : {f1:.4f}")

    return acc, prec, rec, f1

In [ ]:
# Link to drive with images for reference
# https://drive.google.com/drive/folders/1WIZB_ZLItOE0XkjQg2FsLZIuoSnIcYQp
!apt-get install -y p7zip-full
DRIVE_ZIP_PATH_1 = '/content/drive/MyDrive/Colab Notebooks/RevealAI/48K First Sample/test.7z'
DRIVE_ZIP_PATH_2 = '/content/drive/MyDrive/Colab Notebooks/RevealAI/48K First Sample/train.7z'
DRIVE_ZIP_PATH_3 = '/content/drive/MyDrive/Colab Notebooks/RevealAI/48K First Sample/validation.7z'


# # Local temporary disk destination
LOCAL_DESTINATION = '/content/'

# # Execute the copy command

!cp "{DRIVE_ZIP_PATH_1}" "{LOCAL_DESTINATION}"
!cp "{DRIVE_ZIP_PATH_2}" "{LOCAL_DESTINATION}"
!cp "{DRIVE_ZIP_PATH_3}" "{LOCAL_DESTINATION}"

print(f" Copied data to local Colab disk.")
# # Path to the ZIP file on the local disk

LOCAL_ZIP_PATH_1 = '/content/test.7z'
LOCAL_ZIP_PATH_2 = '/content/train.7z'
LOCAL_ZIP_PATH_3 = '/content/validation.7z'

# # Execute the unzip command
# # -q: quiet (less terminal output)
# # -d /content/: extract contents to the /content/ directory

!7z x "{LOCAL_ZIP_PATH_1}" -o/content/
!7z x "{LOCAL_ZIP_PATH_2}" -o/content/
!7z x "{LOCAL_ZIP_PATH_3}" -o/content/
print(" Unzipping complete! Your data is now fast to access.")

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
p7zip-full is already the newest version (16.02+dfsg-8).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.
 Copied data to local Colab disk.

7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,12 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (50657),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/                   1 file, 714238854 bytes (682 MiB)

Extracting archive: /content/test.7z
--
Path = /content/test.7z
Type = 7z
Physical Size = 714238854
Headers Size = 103910
Method = LZMA:23
Solid = +
Blocks = 1

  0%      0% - test/ai/000_biggan_00130.jpg                                     1% 50 - test/ai/054_biggan_00066.jpg            

In [ ]:
# imports
import os, sys,torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os
from PIL import Image
import torch
import torch.nn.functional as F
import timm
import torch.nn as nn
import matplotlib.pyplot as plt
import random
import torch
import torch.nn as nn
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from torchvision.models import shufflenet_v2_x1_0, ShuffleNet_V2_X1_0_Weights

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

Device: cuda


In [ ]:
# u have to get ride of the empty files or the model wont run
import os

folders = [
    "/content/train/ai",
    "/content/train/real",
    "/content/test/ai",
    "/content/test/real",
    "/content/validation/ai",
    "/content/validation/real"
]

count_removed = 0
for folder in folders:
  for f in os.listdir(folder):
    path = os.path.join(folder, f)
    if os.path.isfile(path) and f.lower().endswith(('.jpg', '.jpeg')):
        if os.path.getsize(path) == 0:
            os.remove(path)
            count_removed += 1
            # print(f"Deleted empty file: {path}")
print('Count of deleted files : ', count_removed)


Count of deleted files :  0


In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

train_path = "/content/train"
val_path  = "/content/validation"
test_path = "/content/test"

train_data = datasets.ImageFolder(train_path, transform=transform)
val_data  = datasets.ImageFolder(val_path,  transform=transform)
test_data = datasets.ImageFolder(test_path,  transform=transform)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader  = DataLoader(val_data, batch_size=32, shuffle=False)
test_loader = DataLoader(test_data,  batch_size=32, shuffle=False)

class_names = train_data.classes
num_classes = len(class_names)
num_classes

2

Count of deleted files :  68


In [ ]:
test_model(
    model_path="/content/drive/MyDrive/resnet50_scaled.pth",
    model_constructor=lambda: timm.create_model("resnet50", pretrained=False, num_classes=2),
    model_name="ResNet50"
)

Test Accuracy : 0.9261
Test Precision: 0.9274
Test Recall   : 0.9261
Test F1 Score : 0.9261


(0.9261165994156115,
 0.9273680471747765,
 0.9261165994156115,
 0.9260585345472363)

In [ ]:
test_model(
    model_path="/content/drive/MyDrive/Shufflenet_model_best_scaled.pth",
    model_constructor=lambda: shufflenet_v2_x1_0(weights=None, num_classes=2),
    model_name="ShuffleNetV2"
)

Test Accuracy : 0.4689
Test Precision: 0.4453
Test Recall   : 0.4689
Test F1 Score : 0.4064


(0.46890218449979126,
 0.4453471356322028,
 0.46890218449979126,
 0.40639573788798217)

In [ ]:

# Link to drive with images for reference
# https://drive.google.com/drive/folders/1WIZB_ZLItOE0XkjQg2FsLZIuoSnIcYQp
!apt-get install -y p7zip-full
DRIVE_ZIP_PATH_1 = '/content/drive/MyDrive/Colab Notebooks/RevealAI/48K Cropped/test.7z'
DRIVE_ZIP_PATH_2 = '/content/drive/MyDrive/Colab Notebooks/RevealAI/48K Cropped/train.7z'
DRIVE_ZIP_PATH_3 = '/content/drive/MyDrive/Colab Notebooks/RevealAI/48K Cropped/validation.7z'


# # Local temporary disk destination
LOCAL_DESTINATION = '/content/'

# # Execute the copy command

!cp "{DRIVE_ZIP_PATH_1}" "{LOCAL_DESTINATION}"
!cp "{DRIVE_ZIP_PATH_2}" "{LOCAL_DESTINATION}"
!cp "{DRIVE_ZIP_PATH_3}" "{LOCAL_DESTINATION}"

print(f" Copied data to local Colab disk.")
# # Path to the ZIP file on the local disk

LOCAL_ZIP_PATH_1 = '/content/test.7z'
LOCAL_ZIP_PATH_2 = '/content/train.7z'
LOCAL_ZIP_PATH_3 = '/content/validation.7z'

# # Execute the unzip command
# # -q: quiet (less terminal output)
# # -d /content/: extract contents to the /content/ directory

!7z x "{LOCAL_ZIP_PATH_1}" -o/content/
!7z x "{LOCAL_ZIP_PATH_2}" -o/content/
!7z x "{LOCAL_ZIP_PATH_3}" -o/content/
print(" Unzipping complete! Your data is now fast to access.")

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
p7zip-full is already the newest version (16.02+dfsg-8).
0 upgraded, 0 newly installed, 0 to remove and 41 not upgraded.
 Copied data to local Colab disk.

7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,12 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (50657),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/                   1 file, 341717408 bytes (326 MiB)

Extracting archive: /content/test.7z
--
Path = /content/test.7z
Type = 7z
Physical Size = 341717408
Headers Size = 95582
Method = LZMA2:24 BCJ
Solid = +
Blocks = 1

  0%      0% 64 - DL_Temp/test/ai/063_sdv5_00106_crop1.jpg                                                    1% 130 - DL_Temp/test/

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

train_path = "/content/DL_Temp/train"
val_path  = "/content/DL_Temp/validation"
test_path = "/content/DL_Temp/test"

train_data = datasets.ImageFolder(train_path, transform=transform)
val_data  = datasets.ImageFolder(val_path,  transform=transform)
test_data = datasets.ImageFolder(test_path,  transform=transform)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader  = DataLoader(val_data, batch_size=32, shuffle=False)
test_loader = DataLoader(test_data,  batch_size=32, shuffle=False)

class_names = train_data.classes
num_classes = len(class_names)
num_classes

2

In [ ]:
test_model(
    model_path="/content/drive/MyDrive/resnet50_scaled.pth",
    model_constructor=lambda: timm.create_model("resnet50", pretrained=False, num_classes=2),
    model_name="ResNet50"
)

Test Accuracy : 0.7750
Test Precision: 0.8149
Test Recall   : 0.7750
Test F1 Score : 0.7675


(0.7750104355085571,
 0.8149379900271596,
 0.7750104355085571,
 0.7675483146825033)

In [ ]:
test_model(
    model_path="/content/drive/MyDrive/Shufflenet_model_best_scaled.pth",
    model_constructor=lambda: shufflenet_v2_x1_0(weights=None, num_classes=2),
    model_name="ShuffleNetV2"
)

Test Accuracy : 0.4571
Test Precision: 0.4337
Test Recall   : 0.4571
Test F1 Score : 0.4057


(0.45707527480172533,
 0.43366703032581155,
 0.45707527480172533,
 0.4057029417630315)